In [1]:
import os
import pandas as pd
import shutil
import re
import pprint
import math
import numpy as np

from ucsbdistrict.inputs import CUP_inputs, weather_df, heatPump_df,districtModule_inputs,TES_inputs,dateTime_df,coldTank_schedule2_inputs_df,hotTank_schedule1_inputs_df,\
districtModule_inputs


inputs 293738.9131106457


In [2]:
# Load the DataFrame from the pickle file
buildingMixing_output = pd.read_pickle('buildingMixing_output.pkl')

# Now 'df' contains the DataFrame loaded from the pickle file
buildingMixing_output.head()  # Verify the DataFrame

,Total Space Heating Load (Btu/h),Total DHW Load (Btu/h),Total Heating Load (Btu/h),District HWS Flow (gpm),District HWRT (°F),District CHWS Flow (gpm),District CHWRT (°F),Total Cooling Load (Btu/h)
0,1.231706e+07,3.827353e+06,1.614441e+07,520.917695,103.0155,0.024464,54.000009,122.322571
1,9.027328e+06,3.236478e+06,1.226381e+07,389.913673,102.094763,0.174641,54.000061,873.211983
2,1.029800e+07,3.468767e+06,1.376677e+07,440.544528,102.501126,0.174641,54.000061,873.211983
3,1.133739e+07,3.165227e+06,1.450262e+07,472.558806,103.620896,0.174641,54.000061,873.211983
4,1.177647e+07,4.181198e+06,1.595766e+07,508.396115,102.223504,0.174641,54.000061,873.211983


In [3]:
# ##Heat pump input

# districtHWRT = buildingMixing_output["District HWRT (°F)"]
# districtCHWRT = buildingMixing_output["District CHWRT (°F)"]
# districtHWSflow = buildingMixing_output["District HWS Flow (gpm)"]
# districtCHWSflow = buildingMixing_output["District CHWS Flow (gpm)"]
# HP_leavingHW = heatPump_df["Leaving Hot Water (°F)"]
# HP_enteringHW = heatPump_df["Entering Hot Water (°F)"]
# HP_enteringCHW = heatPump_df["Entering Cold Water (°F)"]
# HP_leavingCHW = heatPump_df["Leaving Cold Water (°F)"]
# HP_maxHeating = heatPump_df["Max Heating Output (Btu/h)"]
# HP_maxCooling = heatPump_df["Max Cooling Output (Btu/h)"]
# HW_districtSTP =  districtModule_inputs["Loop HW STP (°F)"]
# HP_heatingCapacity=CUP_inputs["Heat Pumps"]["Heating Capacity"]
# HP_coolingCapacity=CUP_inputs["Heat Pumps"]["Cooling Capacity"]
# HP_waterSource=CUP_inputs["Heat Pumps"]["Water-Source Only %"]
# HP_airWaterSource=CUP_inputs["Heat Pumps"]["Air-Source + Water Source %"]





In [4]:
# from ucsbdistrict.heatPump import heatPump

# heatPump_instance = heatPump(districtHWRT=districtHWRT,districtCHWRT=districtCHWRT,HW_districtSTP = HW_districtSTP,\
#                              HP_leavingHW=HP_leavingHW,HP_enteringHW=HP_enteringHW,HP_enteringCHW=HP_enteringCHW,HP_leavingCHW=HP_leavingCHW,\
#                               HP_maxHeating=HP_maxHeating,HP_maxCooling=HP_maxCooling )

In [5]:
# heatPump_output_df = heatPump_instance.compute()

In [6]:
# print(HP_enteringHW - districtHWRT)

In [7]:
#TES inputs

HW_upperTankVol = TES_inputs["Initial Conditions"]["TES Hot Tank Upper Tank Volume"]
HW_lowerTankVol = TES_inputs["Initial Conditions"]["TES Hot Tank Lower Tank Volume"]
HW_TankVol = TES_inputs["Tank Properties"]["TES Hot Tank Volume"]
HW_wallLoss = TES_inputs["Tank Properties"]["TES Hot Wall-Losses"]
HW_ThermocloneLoss = TES_inputs["Tank Properties"]["TES Hot Thermocline/other-Losses"]
CHW_upperTankVol = TES_inputs["Initial Conditions"]["TES Cold Tank Upper Tank Volume"]
CHW_lowerTankVol = TES_inputs["Initial Conditions"]["TES Cold Tank Lower Tank Volume"]
CHW_TankVol = TES_inputs["Tank Properties"]["TES Cold Tank Volume"]
CHW_wallLoss = TES_inputs["Tank Properties"]["TES Cold Wall-Losses"]
CHW_ThermocloneLoss = TES_inputs["Tank Properties"]["TES Cold Thermocline/other-Losses"]

#parameters for second instance
thermoclineDischarge = TES_inputs["Tank Charging Characteristics"]["Fraction of Thermocline to discharge"]
stopCharging = TES_inputs["Tank Charging Characteristics"]["Stop Charging tank at ____ Capacity"]
totalHeating_load=buildingMixing_output["Total Heating Load (Btu/h)"]
#HP_heatingCapacity taken from heat pump inputs
districtHWRT = buildingMixing_output["District HWRT (°F)"]
hotMaxflow = TES_inputs["Tank Charging Characteristics"]["TES Hot Max Flow"]
coldMaxflow = TES_inputs["Tank Charging Characteristics"]["TES Cold Max Flow"]
HW_districtSTP =districtModule_inputs["Loop HW STP (°F)"]
HW_upperTankTemp = TES_inputs["Initial Conditions"]["TES Hot Tank Upper Tank Temp"]
HW_lowerTankTemp = TES_inputs["Initial Conditions"]["TES Hot Tank Lower Tank Temp"]
HP_heatingCapacity=CUP_inputs["Heat Pumps"]["Heating Capacity"]

##cold tank inputs
totalCooling_load=buildingMixing_output["Total Cooling Load (Btu/h)"]
districtCHWRT = buildingMixing_output["District CHWRT (°F)"]
CHW_lowerTankTemp = TES_inputs["Initial Conditions"]["TES Cold Tank Lower Tank Temp"]
CHW_upperTankTemp = TES_inputs["Initial Conditions"]["TES Cold Tank Upper Tank Temp"]
# HP_CHWST = heatPump_output_df["HP CHWST (°F)"]



In [8]:
#TES instance

# from ucsbdistrict.TES import TES

# TES_instance_one = TES(HW_upperTankVol=HW_upperTankVol,HW_TankVol=HW_TankVol,HW_wallLoss=HW_wallLoss,HW_ThermocloneLoss=HW_ThermocloneLoss,HW_lowerTankVol=HW_lowerTankVol,\
#           CHW_upperTankVol=CHW_upperTankVol,CHW_lowerTankVol=CHW_lowerTankVol,CHW_TankVol=CHW_TankVol,CHW_wallLoss=CHW_wallLoss,CHW_ThermocloneLoss=CHW_ThermocloneLoss,\
#             HP_heatingCapacity=HP_heatingCapacity,thermoclineDischarge=thermoclineDischarge,stopCharging=stopCharging,totalHeating_load=totalHeating_load,HW_districtSTP=HW_districtSTP,\
#               districtHWRT=districtHWRT,hotMaxflow=hotMaxflow,coldMaxflow=coldMaxflow,HW_upperTankTemp=HW_upperTankTemp)

In [9]:
# TES_instance_one.compute()

In [10]:
#print TES inputs for checking
print(TES_inputs)

{'Conversion Rates': {'CUF to GAL': 7.48, 'GAL to Lbs': 8.34}, 'Initial Conditions': {'Starting TES Hot Charge': 0.95, 'TES Hot Tank Upper Tank Temp': 170.0, 'TES Hot Tank Lower Tank Temp': 80.0, 'TES Hot Tank Upper Tank Volume': 251146.77070960205, 'TES Hot Thermocline thickness': 2.0, 'TES Hot Thermocline volume': 29373.891311064566, 'TES Hot Tank Lower Tank Volume': 13218.25108997908, 'Starting TES Cold Charge': 0.95, 'TES Cold Tank Upper Tank Temp': 60.0, 'TES Cold Tank Lower Tank Temp': 44.0, 'TES Cold Tank Upper Tank Volume': 61332.68505750294, 'TES Cold Thermocline thickness': 2.0, 'TES Cold Thermocline volume': 42298.40348793298, 'TES Cold Tank Lower Tank Volume': 1165321.0160925535}, 'Tank Charging Characteristics': {'TES Hot Capacity': 220480.4281808507, 'TES Cold Capacity': 169328.96884289332, 'TES Hot Max % of Total Tank Volume moved in 1 hr': 0.08, 'TES Cold Max % of Total Tank Volume moved in 1 hr': 0.08, 'TES Hot Max Flow': 395, 'TES Cold Max Flow': 1695, 'Fraction of Th

In [11]:
#CUP inputs
timeStamp = dateTime_df
hotTank_schedule = hotTank_schedule1_inputs_df 
coldTank_schedule = coldTank_schedule2_inputs_df

totalCooling_load=buildingMixing_output["Total Cooling Load (Btu/h)"]
hotTank_schedule.head()

,Schedules,Load Shift Control,Month,0,1,2,3,4,5,6,...,15,16,17,18,19,20,21,22,23,Total
0,Schedules,Load Shift Control - Hot Tank by month,1,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,13
1,Schedules,Load Shift Control - Hot Tank by month,2,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,13
2,Schedules,Load Shift Control - Hot Tank by month,3,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,13
3,Schedules,Load Shift Control - Hot Tank by month,4,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,13
4,Schedules,Load Shift Control - Hot Tank by month,5,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,13


In [12]:
type(dateTime_df)
# print(timeStamp)
timeStamp.head()

,Date and Time
0,2010-01-01 00:00:00
1,2010-01-01 01:00:00
2,2010-01-01 02:00:00
3,2010-01-01 03:00:00
4,2010-01-01 04:00:00


In [13]:

from ucsbdistrict.CUP import CUP
CUP_instance = CUP(timeStamp=timeStamp,hotTank_schedule=hotTank_schedule,coldTank_schedule=coldTank_schedule,totalHeating_load=totalHeating_load,\
                   totalCooling_load=totalCooling_load)

In [14]:
CUP_output_df = CUP_instance.compute_CUP()
CUP_output_df.head()

,Hot Load Shift Charging,Cold Load Shift Charging,Hot Load Shift Hours,Cold Load Shift Hours,Shift Count,Predicted Day's Heating Load (Btu/h),Predicted Day's Cooling Load (Btu/h),Predicted Heating Load in Load Shift Window (Btu/h),Predicted Cooling Load in Load Shift Window (Btu/h)
0,False,True,13,9,0,4.658689e+08,6.708789e+07,2.770624e+08,5728.561004
1,False,True,13,9,0,4.658689e+08,6.708789e+07,2.770624e+08,5728.561004
2,False,True,13,9,0,4.658689e+08,6.708789e+07,2.770624e+08,5728.561004
3,False,True,13,9,0,4.658689e+08,6.708789e+07,2.770624e+08,5728.561004
4,False,True,13,9,0,4.658689e+08,6.708789e+07,2.770624e+08,5728.561004


In [15]:
# CUP_output_df.to_pickle('CUP_output_df.pkl')

In [16]:
# HotLoadShiftCharging = CUP_instance["Hot Load Shift Charging"]

# print(HotLoadShiftCharging)

In [17]:
from ucsbdistrict.TES import TES
TES_instance_two = TES(HW_upperTankVol=HW_upperTankVol,HW_TankVol=HW_TankVol,HW_wallLoss=HW_wallLoss,HW_ThermocloneLoss=HW_ThermocloneLoss,HW_lowerTankVol=HW_lowerTankVol,\
                      CHW_upperTankVol=CHW_upperTankVol,CHW_lowerTankVol=CHW_lowerTankVol,CHW_TankVol=CHW_TankVol,CHW_wallLoss=CHW_wallLoss,CHW_ThermocloneLoss=CHW_ThermocloneLoss,\
                      HP_heatingCapacity=HP_heatingCapacity,thermoclineDischarge=thermoclineDischarge,stopCharging=stopCharging,CUP_output_df=CUP_output_df,totalHeating_load=totalHeating_load,\
                        HW_districtSTP=HW_districtSTP,districtHWRT=districtHWRT,hotMaxflow=hotMaxflow,coldMaxflow=coldMaxflow,HW_upperTankTemp=HW_upperTankTemp,HW_lowerTankTemp=HW_lowerTankTemp,totalCooling_load=totalCooling_load,\
                      districtCHWRT = districtCHWRT, CHW_lowerTankTemp = CHW_lowerTankTemp, CHW_upperTankTemp=CHW_upperTankTemp)
# TES_instance_two.compute_2()


In [18]:
TES_H_ouput1_df = TES_instance_two.TES_H_calculate()
TES_H_ouput1_df.head(15)

<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


,TES Hot Capacity hr-1 (%),TES Hot Th after losses (Gal)_supply temp,TES Hot Tc after losses (Gal)_return temp,TES Hot Thermocline (Gal),TES Hot Status,Hot Load-Shift Charge Rate (gpm),Flow into TES Hot Th (gpm),TES Hot Th Previous Hour (°F),TES Flow to meet Campus Heating (gpm),Flow out of TES Hot Th (gpm),TES Hot Th (°F),TES Hot Th (Gal),TES Hot Tc (Gal),TES Hot Capacity (%),districtHWRT,TES Hot Tc Previous Hour(°F),TES Hot Tc (°F)
0,0.855000,251071.42667838917,13214.285614652086,29453.200818,Discharge,0.0,0,170.0,482.034245,395.0,170.0,227371.426678,36914.285615,0.774060,103.0155,80.000000,94.776593
1,0.774060,227303.21525038563,36903.21132896769,29532.486531,Discharge,0.0,0,170.0,361.203540,361.20354019660437,170.0,205631.002839,58575.423741,0.700047,102.094763,94.776593,97.484229
2,0.700047,205178.61463234448,58446.55780853428,30113.740670,Discharge,0.0,0,170.0,407.911055,395.0,170.0,181478.614632,82146.557809,0.617823,102.501126,97.484229,98.964994
3,0.617823,181079.36168015332,81965.8353813555,30693.716049,Discharge,0.0,0,170.0,436.963356,395.0,170.0,157379.361680,105665.835381,0.535780,103.620896,98.964994,100.093330
4,0.535780,157033.127084457,105433.37054351652,31272.415483,Discharge,0.0,0,170.0,470.890774,395.0,170.0,133333.127084,129133.370544,0.453917,102.223504,100.093330,100.543522
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.331745,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.920655,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.0415,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.192218,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.654645,NaN,NaN


In [19]:
##Heat pump input

districtHWRT = buildingMixing_output["District HWRT (°F)"]
districtCHWRT = buildingMixing_output["District CHWRT (°F)"]
districtHWSflow = buildingMixing_output["District HWS Flow (gpm)"]
districtCHWSflow = buildingMixing_output["District CHWS Flow (gpm)"]
HP_leavingHW = heatPump_df["Leaving Hot Water (°F)"]
HP_enteringHW = heatPump_df["Entering Hot Water (°F)"]
HP_enteringCHW = heatPump_df["Entering Cold Water (°F)"]
HP_leavingCHW = heatPump_df["Leaving Cold Water (°F)"]
HP_maxHeating = heatPump_df["Max Heating Output (Btu/h)"]
HP_maxCooling = heatPump_df["Max Cooling Output (Btu/h)"]
HW_districtSTP =  districtModule_inputs["Loop HW STP (°F)"]
HP_heatingCapacity=CUP_inputs["Heat Pumps"]["Heating Capacity"]
HP_coolingCapacity=CUP_inputs["Heat Pumps"]["Cooling Capacity"]
HP_waterSource=CUP_inputs["Heat Pumps"]["Water-Source Only %"]
HP_airWaterSource=CUP_inputs["Heat Pumps"]["Air-Source + Water Source %"]


In [20]:
#Heat pump inputs from TES

TES_H_shiftChargeRate = TES_H_ouput1_df["Hot Load-Shift Charge Rate (gpm)"]
TES_H_flowOut = TES_H_ouput1_df["Flow out of TES Hot Th (gpm)"]



In [21]:
from ucsbdistrict.heatPump import heatPump

heatPump_instance = heatPump(districtHWRT=districtHWRT,districtCHWRT=districtCHWRT,HW_districtSTP = HW_districtSTP,districtHWSflow=districtHWSflow,\
                            HP_leavingHW=HP_leavingHW,HP_enteringHW=HP_enteringHW,HP_enteringCHW=HP_enteringCHW,HP_leavingCHW=HP_leavingCHW,\
                            HP_maxHeating=HP_maxHeating,HP_maxCooling=HP_maxCooling,TES_H_shiftChargeRate=TES_H_shiftChargeRate,\
                            TES_H_flowOut=TES_H_flowOut,HP_heatingCapacity=HP_heatingCapacity,HP_coolingCapacity=HP_coolingCapacity)

In [22]:
heatPump_output_df = heatPump_instance.compute()
heatPump_output_df.head(15)

ppp 0       96.1
1       96.1
2       96.1
3       96.1
4       96.1
        ... 
8755    96.1
8756    96.1
8757    96.1
8758    96.1
8759    96.1
Name: Max Cooling Output (Btu/h), Length: 8760, dtype: float64
value1 0       4217266.902527
1        974784.181853
2       1537102.168396
3       2574142.022344
4       3842795.657386
             ...      
8755               NaN
8756               NaN
8757               NaN
8758               NaN
8759               NaN
Length: 8760, dtype: object self.HP_heatingCapacity 70000000.0
value1 0       4217266.902527
1        974784.181853
2       1537102.168396
3       2574142.022344
4       3842795.657386
             ...      
8755               NaN
8756               NaN
8757               NaN
8758               NaN
8759               NaN
Length: 8760, dtype: object self.HP_heatingCapacity 70000000.0
ppp 0       96.1
1       96.1
2       96.1
3       96.1
4       96.1
        ... 
8755    96.1
8756    96.1
8757    96.1
8758    96.1
8759    96

c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)


cc 1.5150884495317378
ppp 0       96.1
1       96.1
2       96.1
3       96.1
4       96.1
        ... 
8755    96.1
8756    96.1
8757    96.1
8758    96.1
8759    96.1
Name: Max Cooling Output (Btu/h), Length: 8760, dtype: float64
xx float64
value1 0       4217266.902527
1        974784.181853
2       1537102.168396
3       2574142.022344
4       3842795.657386
             ...      
8755               NaN
8756               NaN
8757               NaN
8758               NaN
8759               NaN
Length: 8760, dtype: object self.HP_heatingCapacity 70000000.0
ppp 0       96.1
1       96.1
2       96.1
3       96.1
4       96.1
        ... 
8755    96.1
8756    96.1
8757    96.1
8758    96.1
8759    96.1
Name: Max Cooling Output (Btu/h), Length: 8760, dtype: float64
ccc 0        2783512.014648
1         643384.339808
2        1014529.659223
3        1699004.452935
4        2536350.705184
             ...       
8755    46201923.076923
8756    46201923.076923
8757    46201923.076923
8758

c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)


ppp 0       96.1
1       96.1
2       96.1
3       96.1
4       96.1
        ... 
8755    96.1
8756    96.1
8757    96.1
8758    96.1
8759    96.1
Name: Max Cooling Output (Btu/h), Length: 8760, dtype: float64
cc 1.5150884495317378
ppp 0       96.1
1       96.1
2       96.1
3       96.1
4       96.1
        ... 
8755    96.1
8756    96.1
8757    96.1
8758    96.1
8759    96.1
Name: Max Cooling Output (Btu/h), Length: 8760, dtype: float64
xx float64
value1 0       4217266.902527
1        974784.181853
2       1537102.168396
3       2574142.022344
4       3842795.657386
             ...      
8755               NaN
8756               NaN
8757               NaN
8758               NaN
8759               NaN
Length: 8760, dtype: object self.HP_heatingCapacity 70000000.0
ppp 0       96.1
1       96.1
2       96.1
3       96.1
4       96.1
        ... 
8755    96.1
8756    96.1
8757    96.1
8758    96.1
8759    96.1
Name: Max Cooling Output (Btu/h), Length: 8760, dtype: float64
ccc 0        2

c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)


,HP CHWST (°F),Max Unitary Heating Output (Btu/h),Max Unitary Cooling Output (Btu/h),Heat:Cool,Required Heating Output (Btu/h),HWSFLow,self.HW_districtSTP,self.districtHWRT,Possible Recovered CHW (Btu/h),Possible Recovered CHW (gpm)
0,44,145.6,96.1,1.515088,4217266.902527,520.917695,170,103.0155,2783512.014648,556.701926
1,44,145.6,96.1,1.515088,974784.181853,389.913673,170,102.094763,643384.339808,128.67608
2,44,145.6,96.1,1.515088,1537102.168396,440.544528,170,102.501126,1014529.659223,202.90469
3,44,145.6,96.1,1.515088,2574142.022344,472.558806,170,103.620896,1699004.452935,339.798811
4,44,145.6,96.1,1.515088,3842795.657386,508.396115,170,102.223504,2536350.705184,507.267036
5,44,145.6,96.1,1.515088,70000000.0,479.779617,170,99.331745,46201923.076923,9240.320138
6,44,145.6,96.1,1.515088,70000000.0,509.883985,170,95.920655,46201923.076923,9240.312215
7,44,145.6,96.1,1.515088,70000000.0,521.839294,170,93.0415,46201923.076923,9240.304293
8,44,145.6,96.1,1.515088,70000000.0,520.53427,170,91.192218,46201923.076923,9240.360848
9,44,145.6,96.1,1.515088,70000000.0,685.206776,170,94.654645,46201923.076923,9068.773702


In [23]:
HP_possRecCHW=heatPump_output_df["Possible Recovered CHW (gpm)"]
HP_CHWST=heatPump_output_df["HP CHWST (°F)"]
TES_hotCapacityHr =TES_H_ouput1_df["TES Hot Capacity hr-1 (%)"]

In [24]:
TES_instance_three = TES(HW_upperTankVol=HW_upperTankVol,HW_TankVol=HW_TankVol,HW_wallLoss=HW_wallLoss,HW_ThermocloneLoss=HW_ThermocloneLoss,HW_lowerTankVol=HW_lowerTankVol,\
                      CHW_upperTankVol=CHW_upperTankVol,CHW_lowerTankVol=CHW_lowerTankVol,CHW_TankVol=CHW_TankVol,CHW_wallLoss=CHW_wallLoss,CHW_ThermocloneLoss=CHW_ThermocloneLoss,\
                      HP_heatingCapacity=HP_heatingCapacity,thermoclineDischarge=thermoclineDischarge,stopCharging=stopCharging,CUP_output_df=CUP_output_df,totalHeating_load=totalHeating_load,\
                        HW_districtSTP=HW_districtSTP,districtHWRT=districtHWRT,hotMaxflow=hotMaxflow,coldMaxflow=coldMaxflow,HW_upperTankTemp=HW_upperTankTemp,HW_lowerTankTemp=HW_lowerTankTemp,totalCooling_load=totalCooling_load,\
                      districtCHWRT = districtCHWRT, CHW_lowerTankTemp = CHW_lowerTankTemp, CHW_upperTankTemp=CHW_upperTankTemp,HP_possRecCHW=HP_possRecCHW,HP_CHWST=HP_CHWST, TES_hotCapacityHr=TES_hotCapacityHr)
# TES_instance_two.compute_2()


In [25]:
TES_C_ouput1_df = TES_instance_three.TES_C_calculate()
TES_C_ouput1_df.head()

trial 44.0 44.0 1198488.5995572987 1268952.1046379893
trial 43.99999999999999 44.0 1205958.9881818262 1268952.1046379893
trial 44.00000000000001 44.0 1217640.4075050652 1268952.1046379893
trial 44.00000000000001 44.0 1237530.8015136071 1268952.1046379893
trial 44.0 44.0 1237035.7891930018 1268952.1046379893


,TES Cold Capacity hr-1 (%),TES Cold Th after losses (Gal)_supply temp,TES Cold Tc after losses (Gal)_return temp,TES Cold Thermocline (Gal),TES Cold Charge Window Adjustment,TES Cold Status,Cold Load-Shift Charge Rate (gpm),TES Cold Tc Previous Hour (°F),TES Flow to meet Campus Heating (gpm),HP Flow to meet Campus Cooling (gpm),Flow into TES Cold Tc (gpm),Flow out of TES Cold Tc (gpm),TES Cold Th (Gal),TES Cold Tc (Gal),TES Cold Tc (°F),TES Cold Capacity (%)
0,0.918333,61320.41852049144,1165087.951889335,42543.734228,False,Charge,556.677461132728,44.0,0.024464,0.024464,556.677461132728,0.0,27919.770853,1.198489e+06,44.0,0.944471
1,0.944471,27914.18689835725,1198248.9018373874,42789.015902,False,Charge,128.50143907397953,44.0,0.174641,0.174641,128.50143907397953,0.0,20204.100554,1.205959e+06,44.0,0.950358
2,0.950358,20196.01891369691,1205476.6045865535,43279.481138,False,Charge,202.73004864186316,44.0,0.174641,0.174641,202.73004864186316,0.0,8032.215995,1.217640e+06,44.0,0.959564
3,0.959564,8029.003108787045,1217153.3513420632,43769.750187,False,Charge,339.62416952573454,44.0,0.174641,0.174641,339.62416952573454,0.0,0.000000,1.237531e+06,44.0,0.975238
4,0.975238,0.0,1237035.7891930018,31916.315445,False,Hold,0.0,44.0,0.174641,0.174641,0.0,0.0,0.000000,1.237036e+06,44.0,0.974848
